In [213]:
import pandas as pd
from pycountry import countries


In [214]:
out_dir = '.\\out\\'
pickle_out_file = 'cleaned_world_bank_data.pkl'
train_regressors_out_file = 'train_regressors_data.pkl'
train_targets_out_file = 'train_targets_data.pkl'
test_targets_out_file = 'test_targets_data.pkl'

In [215]:
world_bank_file_out = "world_bank_data"
wb_data = pd.read_pickle(world_bank_file_out)

In [216]:
#Create list of valid countries taken from the pycountry library
#This gives us a list of 249 coutries that we will focus on
countries_list = []
for country in countries:
    countries_list.append(country.name)

*Let's have a look at regions that are used in the world bank data that are not in the pycountry list:*

In [217]:
exclude_list = [wb_country for wb_country in wb_data.index.levels[0].values if wb_country not in countries_list]
print(exclude_list)

['Arab World', 'Bahamas, The', 'Bolivia', 'British Virgin Islands', 'Caribbean small states', 'Central Europe and the Baltics', 'Channel Islands', 'Congo, Dem. Rep.', 'Congo, Rep.', "Cote d'Ivoire", 'Curacao', 'Czech Republic', 'Early-demographic dividend', 'East Asia & Pacific', 'East Asia & Pacific (IDA & IBRD countries)', 'East Asia & Pacific (excluding high income)', 'Egypt, Arab Rep.', 'Eswatini', 'Euro area', 'Europe & Central Asia', 'Europe & Central Asia (IDA & IBRD countries)', 'Europe & Central Asia (excluding high income)', 'European Union', 'Fragile and conflict affected situations', 'Gambia, The', 'Heavily indebted poor countries (HIPC)', 'High income', 'Hong Kong SAR, China', 'IBRD only', 'IDA & IBRD total', 'IDA blend', 'IDA only', 'IDA total', 'Iran, Islamic Rep.', 'Korea, Dem. People’s Rep.', 'Korea, Rep.', 'Kosovo', 'Kyrgyz Republic', 'Lao PDR', 'Late-demographic dividend', 'Latin America & Caribbean', 'Latin America & Caribbean (excluding high income)', 'Latin Americ

*Clearly, some of the regions in the list above seem to be countries of interest so we will investigate these further now to see why that are not in the pycountry list*

To investigate:  
Bahamas, The  
Bolivia  
British Virgin Islands  
Congo, Dem. Rep.  
Congo, Rep.  
Cote d'Ivoire  
Curacao  
Czech Republic  
Egypt, Arab Rep.  
Eswatini  
Gambia, The  
Hong Kong SAR, China  
Iran, Islamic Rep.  
Korea, Dem. People’s Rep.  
Korea, Rep.  
Kosovo  
Kyrgyz Republic  
Lao PDR  
Macao SAR, China  
Micronesia, Fed. Sts.  
Moldova  
North Macedonia  
Slovak Republic  
St. Kitts and Nevis  
St. Lucia  
St. Martin (French part)  
St. Vincent and the Grenadines  
Tanzania  
Venezuela, RB  
Vietnam  
Virgin Islands (U.S.)  
West Bank and Gaza  
Yemen, Rep.  

*After a bit of investigation all of the above seem to fall into 3 categories:*

1. Countries that have different names in both lists and the name in the World Bank dataset is clearer or simpler.
2. Countries that have different names in both lists and the name in the pycountry library is decided to be the standard for all further processing  (most of the above)
3. 'West Bank and Gaza' and 'Kosovo' are not listed in the pycountry list. I will add these to the countries_list seperately.

*1. Countries that have different names in both lists and the name in the World Bank dataset is clearer or simpler*

In [218]:
#Some of the names in the pycountry list could be simplified or made clearer.
pycountry_list_transform={
#Remove names                      #Replace with these
'Bolivia, Plurinational State of' :'Bolivia',
'Côte d\'Ivoire'                  :'Cote d\'Ivoire',
'Curaçao'                         :'Curacao',
'Czechia'                         :'Czech Republic',
'Viet Nam'                        :'Vietnam'
}

#remove the old names specifie above dictinary
countries_list = [c for c in countries_list if c not in list(pycountry_list_transform.keys()) ]

#Add the new names
countries_list.extend(list(pycountry_list_transform.values()))

*2. Countries that have different names in both lists and the name in the pycountry library is decided to be the standard for all further processing (most of the above)*

In [219]:
countries_transform = {
'Bahamas, The'                  :'Bahamas',
'British Virgin Islands'        :'Virgin Islands, British',
'Congo, Dem. Rep.'              :'Congo, The Democratic Republic of the',
'Congo, Rep.'                   :'Congo',
'Egypt, Arab Rep.'              :'Egypt',
'Eswatini'                      :'Swaziland',
'Gambia, The'                   :'Gambia',
'Hong Kong SAR, China'          :'Hong Kong',
'Iran, Islamic Rep.'            :'Iran, Islamic Republic of',
'Korea, Dem. People’s Rep.'     :'Korea, Democratic People\'s Republic of',
'Korea, Rep.'                   :'Korea, Republic of',
'Kyrgyz Republic'               :'Kyrgyzstan',
'Lao PDR'                       :'Lao People\'s Democratic Republic',
'Macao SAR, China'              :'Macao',
'Micronesia, Fed. Sts.'         :'Micronesia, Federated States of',
'Moldova'                       :'Moldova, Republic of',
'North Macedonia'               :'Macedonia, Republic of',
'Slovak Republic'               :'Slovakia',
'St. Kitts and Nevis'           :'Saint Kitts and Nevis',
'St. Lucia'                     :'Saint Lucia',
'St. Martin (French part)'      :'Saint Martin (French part)',
'St. Vincent and the Grenadines':'Saint Vincent and the Grenadines',
'Tanzania'                      :'Tanzania, United Republic of',
'Venezuela, RB'                 :'Venezuela, Bolivarian Republic of',
'Virgin Islands (U.S.)'         :'Virgin Islands, U.S.',
'Yemen, Rep.'                   :'Yemen'}
    

In [220]:
#Get index as a list, make changes and reapply the list to the World Bank dataframe
index_of_wb_countries = list(wb_data.index.levels[0])

for remove_region, replace_with_region in countries_transform.items():
    index_of_wb_countries[index_of_wb_countries.index(remove_region)] = replace_with_region

#reapply the index to the original dataframe
wb_data.index = wb_data.index.set_levels(index_of_wb_countries, level=0)

*3. 'West Bank and Gaza' and 'Kosovo' are not listed in the pycountry list. I will add these to the countries_list seperately.*

In [221]:
add_list = ['West Bank and Gaza', 'Kosovo']

countries_list.extend(add_list)

*Have another look at what regions are going to be excluded:*

In [222]:
new_exclude_list = [wb_country for wb_country in wb_data.index.levels[0].values if wb_country not in countries_list]
print(new_exclude_list)

['Arab World', 'Caribbean small states', 'Central Europe and the Baltics', 'Channel Islands', 'Early-demographic dividend', 'East Asia & Pacific', 'East Asia & Pacific (IDA & IBRD countries)', 'East Asia & Pacific (excluding high income)', 'Euro area', 'Europe & Central Asia', 'Europe & Central Asia (IDA & IBRD countries)', 'Europe & Central Asia (excluding high income)', 'European Union', 'Fragile and conflict affected situations', 'Heavily indebted poor countries (HIPC)', 'High income', 'IBRD only', 'IDA & IBRD total', 'IDA blend', 'IDA only', 'IDA total', 'Late-demographic dividend', 'Latin America & Caribbean', 'Latin America & Caribbean (excluding high income)', 'Latin America & the Caribbean (IDA & IBRD countries)', 'Least developed countries: UN classification', 'Low & middle income', 'Low income', 'Lower middle income', 'Middle East & North Africa', 'Middle East & North Africa (IDA & IBRD countries)', 'Middle East & North Africa (excluding high income)', 'Middle income', 'North

*Now that we have built up our coutries_list and refactored the World Bank dataframes country index, it is time to apply the filter to the World bank dataframe.*

In [223]:
include_list = countries_list

In [224]:
wb_data_countries_only = wb_data.loc[include_list]

In [225]:
#A little bit of dataframe butchery here!!
#For some reason after the dataframe subset creation above, the index of the new..
#dataframe was the same as the original. Below is a way of getting around this..
#(i.e. by generating a new one using reset_index/set_index)
temp_data = wb_data_countries_only.reset_index()
filtered_data = temp_data.set_index(['country', 'year'])

In [226]:
filtered_data.index.levels[0]

Index(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra',
       'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       ...
       'Uzbekistan', 'Vanuatu', 'Venezuela, Bolivarian Republic of', 'Vietnam',
       'Virgin Islands, British', 'Virgin Islands, U.S.', 'West Bank and Gaza',
       'Yemen', 'Zambia', 'Zimbabwe'],
      dtype='object', name='country', length=216)

In [227]:
filtered_data.shape

(10152, 9)

*Summary of Missing Data:*

In [228]:
filtered_data.isna().sum()/filtered_data.shape[0]

SI.POV.DDAY          0.852935
SI.POV.GINI          0.853625
EN.POP.SLUM.UR.ZS    0.957053
SI.SPR.PC40          0.981678
SE.PRM.UNER          0.600177
SE.XPD.TOTL.GD.ZS    0.638987
SL.TLF.TOTL.IN       0.469661
NY.GDP.MKTP.KD.ZG    0.218282
SP.URB.TOTL.IN.ZS    0.034476
dtype: float64

In [229]:
#According to pandas docs on multiIndex usage: For objects to be indexed and sliced effectively, they need to be sorted.
filtered_data = filtered_data.sort_index()

### Reorganise the data into overlapping window frames

For consumption by the machine forecasting algo we reorganise the dataframe to be a 3 layer multilevel index dataframe: Country, Frame, Year.

In [230]:
countries_in_orig_data = list(filtered_data.index.levels[0]) 

In [231]:
target = 'SI.POV.DDAY'
regressors = list(filtered_data.columns)
regressors.remove(target)
lag = 5
diff = 1
predict_year=2010
number_of_splits=3


In [232]:
#Create an empty test and training dataframes
regressors_index = pd.MultiIndex(levels=[[],[],[]],
                            codes=[[],[],[]],
                            names=[u'country', u'split', u'year'])
target_index = pd.MultiIndex(levels=[[],[]],
                           codes=[[],[]],
                           names=[u'country', u'split'])
test_index = countries_in_orig_data


columns_spilt = regressors
training_data_regressors = pd.DataFrame(index=regressors_index, columns=columns_spilt)
training_data_targets = pd.DataFrame(index=target_index, columns=[target])
test_data_targets = pd.DataFrame(index=test_index, columns=[target])

In [233]:
#fill out the training dataframes (training_data_regressors and training_data_targets)
for country in countries_in_orig_data:
    test_data_targets.loc[country, target] =  filtered_data.loc[(country,str(predict_year)), target]
    for split in range(1,number_of_splits+1):
        year= predict_year - split
        #Add the target value for the spilt to the test_data 
        training_data_targets.loc[(country, split),:] = filtered_data.loc[(country,str(year)), target]
        for l in range(1,lag+1):
            year = year -1
            training_data_regressors.loc[(country,split,l),:] = filtered_data.loc[(country,str(year)), regressors]  
                        

In [ ]:
#According to pandas docs on multiIndex usage: For objects to be indexed and sliced effectively, they need to be sorted.
training_data_regressors = training_data_regressors.sort_index()
training_data_targets = training_data_targets.sort_index()
test_data_targets = test_data_targets.sort_index()

*note on efficiency: If needed when the datasets get bigger the above 3 loop copy can be made faster.  It is writing overlapping windows from one dataframe to another so unless there is a major overhaul we probably need to hold on to one of the for loops but the other 2 can go.*

*1. (Straightforward) The inner-most for loop can be replaced by copying a block of 'lag' rows in one go.*  
*2. (a bit trickier) The other-most loop (counties) could be replaced by making it inside of the spilt loop and copying one split for all the countries in one pandas copy.*

### Write Files to disk

In [210]:
#Write the original dataset filtered to include only countries
filtered_data.to_pickle(out_dir + pickle_out_file)

In [211]:
#Write the time series window data (ready for the forecasting algos)
training_data_regressors.to_pickle(out_dir + train_regressors_out_file)
training_data_targets.to_pickle(out_dir + train_targets_out_file)
test_data_targets.to_pickle(out_dir + test_targets_out_file)